# Introduction
- This introduction is the easiest way for Python folks to play with Percolate
- We provider the Postgres client using the Docker compose connection parameteres as defaults
- We can learn how to register entities after which everyone else becomes possible

In [ ]:
import percolate
from percolate.services import PostgresService
pg = PostgresService()

## Registering entities which are Agents is the key way to use Percolate
- entity models describe agents completely
- Percolate assumes agents are serializable in the "Agent" model which has structured response, sytem prompt and external function refs
- We register Entities as tables that can store data, as Agents that can allow interaction with data
- Agents are added as structured tables but other indexes are added too; embeddings on fields that need them and graph node registry

In [ ]:
from percolate.models.p8 import Project, Agent, ModelField, LanguageModelApi, Function

#we register the core models - these are added by scripts in install anyway but it illustrates for 'bring your own'
for model in [Project, Agent, ModelField, LanguageModelApi, Function]:
    repo = pg.repository(model)
    repo.register(register_entities=True)

## Percolate configures Langauge models and assumes we can load tokens from the env
- While not recommended in production, for simplicity you might add tokens locally in your database
- The example below registers unique model names - the name is sometimes of the from provider-model if you want to select an API

In [ ]:
from percolate.utils import make_uuid
import os

models = [
    LanguageModelApi(id = make_uuid('gpt-4o-2024-08-06'), name = 'gpt-4o-2024-08-06', scheme='openai', completions_uri='https://api.openai.com/v1/chat/completions', token_env_key='OPENAI_API_KEY', token=os.environ['OPENAI_API_KEY']),
    LanguageModelApi(id = make_uuid('gpt-4o-mini'), name = 'gpt-4o-mini', scheme='openai', completions_uri='https://api.openai.com/v1/chat/completions', token_env_key='OPENAI_API_KEY', token=os.environ['OPENAI_API_KEY']),
    LanguageModelApi(id = make_uuid('cerebras-llama3.1-8b'), name = 'cerebras-llama3.1-8b', model='llama3.1-8b', scheme='openai', completions_uri='https://api.cerebras.ai/v1/chat/completions', token_env_key='CEREBRAS_API_KEY', token=os.environ['CEREBRAS_API_KEY']),
    LanguageModelApi(id = make_uuid('groq-llama-3.3-70b-versatile'), name = 'groq-llama-3.3-70b-versatile', model='llama-3.3-70b-versatile', scheme='openai', completions_uri='https://api.groq.com/openai/v1/chat/completions', token_env_key='GROQ_API_KEY', token=os.environ['GROQ_API_KEY']),
    LanguageModelApi(id = make_uuid('claude-3-5-sonnet-20241022'), name = 'claude-3-5-sonnet-20241022', scheme='anthropic', completions_uri='https://api.anthropic.com/v1/messages', token_env_key='ANTHROPIC_API_KEY',token=os.environ['ANTHROPIC_API_KEY']),
    LanguageModelApi(id = make_uuid('gemini-1.5-flash'), name = 'gemini-1.5-flash', scheme='google', completions_uri='https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent', token_env_key='GEMINI_API_KEY', token=os.environ['GEMINI_API_KEY']),
    LanguageModelApi(id = make_uuid('deepseek-chat'), name = 'deepseek-chat', scheme='openai', completions_uri='https://api.deepseek.com/chat/completions', token_env_key='DEEPSEEK_API_KEY', token=os.environ['DEEPSEEK_API_KEY']),
]

fetched=pg.repository(LanguageModelApi).update_records(models)